In [1]:
import pandas as pd
import numpy as np
import os

# 工作路径改为/quoraDupli
os.chdir(os.path.dirname(os.getcwd()))
print os.getcwd()

D:\Program\PycharmProjects\quoraDupli


In [2]:
# 载入经过预处理的数据
df = pd.read_pickle(os.path.join('data', 'questionVector.pickle'))

In [19]:
from sklearn.model_selection import KFold
from keras.optimizers import Adam
# 将dataframe打乱顺序
df = df.reindex(np.random.permutation(df.index))

# 初始化
num_df = df.shape[0]
X = np.zeros([num_df, 2, 300])  # (404288, 2, 300)
Y = np.zeros([num_df])

b = [a[None,:] for a in list(df['q1Vector'].values)]
q1_feats = np.concatenate(b, axis=0)  # (404288, 300)

b = [a[None,:] for a in list(df['q2Vector'].values)]
q2_feats = np.concatenate(b, axis=0)

X[:,0,:] = q1_feats
X[:,1,:] = q2_feats
Y = df['is_duplicate'].values

del b
del q1_feats
del q2_feats

kf = KFold(n_splits=10)
for train_index, test_index in kf.split(X):
    cv_num = 1
    print '第%d次交叉验证开始... ...' % cv_num
    cv_num += 1
    X_train = X[train_index]
    X_test = X[test_index]
    Y_train = Y[train_index]
    Y_test = Y[test_index]
    
    # 特征向量规范
    X_train_norm = np.zeros_like(X_train)
    d = (np.sum(X_train[:,0,:] ** 2, 1) ** (0.5))
    X_train_norm[:,0,:] = (X_train[:,0,:].T / (d + 1e-8)).T
    d = (np.sum(X_train[:,1,:] ** 2, 1) ** (0.5))
    X_train_norm[:,1,:] = (X_train[:,1,:].T / (d + 1e-8)).T
    
    X_test_norm = np.zeros_like(X_test)
    d = (np.sum(X_test[:,0,:] ** 2, 1) ** (0.5))
    X_test_norm[:,0,:] = (X_test[:,0,:].T / (d + 1e-8)).T
    d = (np.sum(X_test[:,1,:] ** 2, 1) ** (0.5))
    X_test_norm[:,1,:] = (X_test[:,1,:].T / (d + 1e-8)).T
    
    net = create_network(300)
    optimizer = Adam(lr=0.001)
    net.compile(loss=contrastive_loss, optimizer=optimizer)
    for epoch in range(5):
    net.fit([X_train_norm[:,0,:], X_train_norm[:,1,:]], Y_train,
            validation_data=([X_test_norm[:,0,:], X_test_norm[:,1,:]], Y_test),
            batch_size=128, nb_epoch=1, shuffle=True, )
    
    # 预测
    pred = net.predict([X_test_norm[:,0,:], X_test_norm[:,1,:]], batch_size=128)
    te_acc = compute_accuracy(pred, Y_test)
    te_f1 = compute_f1(pred, Y_test)
    print '测试集上正确率: %0.2f%%' % (100 * te_acc)
    print '测试集上F1值: %0.6f' % te_f1
    print '第%d次交叉验证完成。' % cv_num
    break

第1次交叉验证... ...


In [ ]:
from keras.models import Model
from keras.layers import Dense, Lambda, merge, BatchNormalization, Activation, Input, Merge
from keras import backend as K

def create_network(input_dim):
    # 网络初始化
    base_network = create_base_network(input_dim)
    
    input_a = Input(shape=(input_dim,))
    input_b = Input(shape=(input_dim,))
    
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    
    distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])
    
    model = Model(input=[input_a, input_b], output=distance)
    return model

def create_base_network(input_dim):
    
    input = Input(shape=(input_dim, ))
    dense1 = Dense(128)(input)
    bn1 = BatchNormalization()(dense1)
    relu1 = Activation('relu')(bn1)

    dense2 = Dense(128)(relu1)
    bn2 = BatchNormalization()(dense2)
    res2 = merge([relu1, bn2], mode='sum')
    relu2 = Activation('relu')(res2)    

    dense3 = Dense(128)(relu2)
    bn3 = BatchNormalization()(dense3)
    res3 = Merge(mode='sum')([relu2, bn3])
    relu3 = Activation('relu')(res3)   
    
    feats = merge([relu3, relu2, relu1], mode='concat')
    bn4 = BatchNormalization()(feats)

    model = Model(input=input, output=bn4)

    return model

def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

def compute_accuracy(predictions, labels):
    return labels[predictions.ravel() < 0.5].mean()

def compute_f1(predictions, labels):
    return labels[predictions.ravel() < 0.5].mean()

In [4]:
from keras.models import Model
from keras.layers import Dense, Lambda, merge, BatchNormalization, Activation, Input, Merge
from keras import backend as K


def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))


def create_base_network(input_dim):
    
    input = Input(shape=(input_dim, ))
    dense1 = Dense(128)(input)
    bn1 = BatchNormalization()(dense1)
    relu1 = Activation('relu')(bn1)

    dense2 = Dense(128)(relu1)
    bn2 = BatchNormalization()(dense2)
    res2 = merge([relu1, bn2], mode='sum')
    relu2 = Activation('relu')(res2)    

    dense3 = Dense(128)(relu2)
    bn3 = BatchNormalization()(dense3)
    res3 = Merge(mode='sum')([relu2, bn3])
    relu3 = Activation('relu')(res3)   
    
    feats = merge([relu3, relu2, relu1], mode='concat')
    bn4 = BatchNormalization()(feats)

    model = Model(input=input, output=bn4)

    return model


def compute_accuracy(predictions, labels):
    '''
    Compute classification accuracy with a fixed threshold on distances.
    '''
    return labels[predictions.ravel() < 0.5].mean()

def create_network(input_dim):
    # 网络初始化
    base_network = create_base_network(input_dim)
    
    input_a = Input(shape=(input_dim,))
    input_b = Input(shape=(input_dim,))
    
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    
    distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])
    
    model = Model(input=[input_a, input_b], output=distance)
    return model

Using TensorFlow backend.


In [5]:
from keras.optimizers import RMSprop, SGD, Adam
net = create_network(300)

# train
#optimizer = SGD(lr=1, momentum=0.8, nesterov=True, decay=0.004)
optimizer = Adam(lr=0.001)
net.compile(loss=contrastive_loss, optimizer=optimizer)

for epoch in range(50):
    net.fit([X_train_norm[:,0,:], X_train_norm[:,1,:]], Y_train,
          validation_data=([X_test_norm[:,0,:], X_test_norm[:,1,:]], Y_test),
          batch_size=128, nb_epoch=1, shuffle=True, )
    
    # compute final accuracy on training and test sets
    pred = net.predict([X_test_norm[:,0,:], X_test_norm[:,1,:]], batch_size=128)
    te_acc = compute_accuracy(pred, Y_test)
    
#    print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
    print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

/software/home/chenzh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:45: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/software/home/chenzh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/software/home/chenzh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:51: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/software/home/chenzh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:54: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layer

Train on 355773 samples, validate on 48515 samples
Epoch 1/1
355773/355773 [==============================] - 21s - loss: 5.4071 - val_loss: 0.2220
* Accuracy on test set: 57.60%
Train on 355773 samples, validate on 48515 samples
Epoch 1/1
355773/355773 [==============================] - 19s - loss: 0.1852 - val_loss: 0.1752
* Accuracy on test set: 63.85%
Train on 355773 samples, validate on 48515 samples
Epoch 1/1
355773/355773 [==============================] - 19s - loss: 0.1658 - val_loss: 0.1643
* Accuracy on test set: 66.40%
Train on 355773 samples, validate on 48515 samples
Epoch 1/1
355773/355773 [==============================] - 20s - loss: 0.1568 - val_loss: 0.1575
* Accuracy on test set: 68.96%
Train on 355773 samples, validate on 48515 samples
Epoch 1/1
355773/355773 [==============================] - 19s - loss: 0.1499 - val_loss: 0.1530
* Accuracy on test set: 69.18%
Train on 355773 samples, validate on 48515 samples
Epoch 1/1
355773/355773 [=============================